In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy.exc import SQLAlchemyError
from sklearn.preprocessing import StandardScaler

from db_queries import username, password, dsn, dbhostname, service_name, dbtables, querys
from table_functions import *

In [2]:
data = {}

try:
    sqlalchemy_engine="oracle+cx_oracle://"+username+":"+password+"@"+dbhostname+"/?service_name="+service_name
    engine = sqlalchemy.create_engine(sqlalchemy_engine, arraysize=1000)
    for table, query in zip(dbtables, querys):
        data.update({table: pd.read_sql(query, engine)})
except SQLAlchemyError as e:
    print(e)

In [ ]:
data = drop_unused_columns(data)
final_table = combine_final_table(data)
final_table = create_final_status(final_table)
final_table = normalize_data(final_table)

In [21]:
pd.set_option('display.max_columns', None)
final_table.head(3)

,czas_fazy_1,czas_fazy_2,czas_fazy_3,max_predkosc,cisnienie_tloka,cisnienie_koncowe,nachdruck_hub,anguss,max_press_kolbenhub,temp_pieca,oni_temp_curr_f1,oni_temp_curr_f2,oni_temp_fore_f1,oni_temp_fore_f2,vds_air_pressure,vds_vac_hose1,vds_vac_hose2,vds_vac_tank,vds_vac_valve1,vds_vac_valve2,czas_taktu,rodzaj_kontroli,assigment_1,assigment_2,assigment_3,assigment_4,assigment_5,assigment_6,assigment_7,assigment_8,assigment_9,assigment_10,assigment_11,assigment_12,assigment_13,assigment_14,assigment_15,assigment_16,assigment_17,assigment_18,assigment_19,assigment_20,assigment_21,assigment_22,assigment_23,assigment_24,assigment_25,assigment_26,assigment_27,assigment_28,flow_1,flow_2,flow_3,flow_4,flow_5,flow_6,flow_7,flow_8,flow_9,flow_10,flow_11,flow_12,flow_13,flow_14,flow_15,flow_16,flow_17,flow_18,flow_19,flow_20,flow_21,flow_22,flow_23,flow_24,flow_25,flow_26,flow_27,flow_28,set_point_1,set_point_2,set_point_3,set_point_4,set_point_5,set_point_6,set_point_7,set_point_8,set_point_9,set_point_10,set_point_11,set_point_12,set_point_13,set_point_14,set_point_15,set_point_16,set_point_17,set_point_18,set_point_19,set_point_20,set_point_21,set_point_22,set_point_23,set_point_24,set_point_25,set_point_26,set_point_27,set_point_28,start_delay_1,start_delay_2,start_delay_3,start_delay_4,start_delay_5,start_delay_6,start_delay_7,start_delay_8,start_delay_9,start_delay_10,start_delay_11,start_delay_12,start_delay_13,start_delay_14,start_delay_15,start_delay_16,start_delay_17,start_delay_18,start_delay_19,start_delay_20,start_delay_21,start_delay_22,start_delay_23,start_delay_24,start_delay_25,start_delay_26,start_delay_27,start_delay_28,working_mode_1,working_mode_2,working_mode_3,working_mode_4,working_mode_5,working_mode_6,working_mode_7,working_mode_8,working_mode_9,working_mode_10,working_mode_11,working_mode_12,working_mode_13,working_mode_14,working_mode_15,working_mode_16,working_mode_17,working_mode_18,working_mode_19,working_mode_20,working_mode_21,working_mode_22,working_mode_23,working_mode_24,working_mode_25,working_mode_26,working_mode_27,working_mode_28,temp_1,temp_2,temp_3,temp_4,temp_5,temp_6,temp_7,temp_8,temp_9,temp_10,temp_11,temp_12,temp_13,temp_14,temp_15,temp_16,temp_17,temp_18,temp_19,temp_20,temp_21,temp_22,temp_23,temp_24,temp_25,temp_26,temp_27,temp_28,id_dmc_DMC,nok_strefa,nok_rodzaj,kod_pola,rodzaj_uszkodzenia,last_machine_number,last_pcf_number,temp_workpiece,temp_hydraulics,pressure_pcf_1,pressure_pcf_2,pressure_pcf_3,machine_nr,cisnienie,przeciek,temperaturatestu,our_final_status
0,2277.0,90.0,45.0,6.30,5.0,282.0,27.0,35.0,2457.0,677.85,79.8,0.0,81.9,-1.0,5713.976,91.37731,90.79861,94.96528,171.2384,130.9028,94.0,NaN,17.0,16.0,17.0,17.0,17.0,17.0,17.0,17.0,16.0,17.0,17.0,2.0,16.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,16.0,17.0,17.0,17.0,17.0,17.0,0.0,0.0,11.1,14.5,10.8,14.3,7.8,9.1,0.0,2.5,9.3,9.8,0.0,10.3,11.5,11.6,11.6,11.6,12.6,11.5,10.6,3.7,0.0,11.1,8.5,15.0,11.6,13.5,10.0,2.0,0.0,15.0,0.0,25.0,0.0,20.0,2.0,15.0,25.0,25.0,2.0,20.0,20.0,25.0,17.0,5.0,0.0,35.0,15.0,5.0,2.0,15.0,0.0,15.0,0.0,15.0,7.0,15.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,5.0,12.0,2.0,5.0,2.0,2.0,5.0,2.0,2.0,2.0,2.0,21.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,3.0,2.0,3.0,2.0,2.0,2.0,2.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,3.0,2.0,85.9,85.7,85.9,96.2,82.7,79.8,80.7,78.7,81.7,72.6,87.1,91.4,81.9,82.4,79.3,79.0,80.4,78.2,82.6,78.6,74.1,78.5,74.8,80.9,80.5,72.3,86.8,77.6,289563.0,0.0,0.0,Brak,Brak,3.0,6.0,27.4,40.200001,101.0,104.0,90.0,3.0,1.061,0.000,31.199999,1.0
1,2220.0,76.0,16.0,5.00,4.0,281.0,0.0,41.0,0.0,0.00,71.8,0.0,70.1,-1.0,4910.590,77.66204,73.61111,74.30556,381.5393,397.5116,85.0,NaN,17.0,16.0,17.0,17.0,17.0,17.0,17.0,17.0,16.0,16.0,17.0,17.0,16.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,16.0,17.0,17.0,17.0,17.0,17.0,0.0,0.0,11.8,15.1,12.1,16.7,7.6,11.6,0.0,0.0,10.1,11.0,0.0,11.6,11.1,13.3,11.5,11.8,12.8,12.8,14.3,4.6,0.0,13.5,9.1,14.3,12.3,15.6,10.0,2.0,10.0,15.0,0.0,25.0,0.0,20.0,2.0,5.0,25.0,

In [ ]:
tables = ['nachdruck_hub', 'anguss', 'max_press_kolbenhub', 'last_machine_number', 'last_pcf_number', 'machine_nr', 'cisnienie_tloka', 'rodzaj_kontroli', 'assigment_10', 'set_point_19',
          'start_delay_3', 'working_mode_5']

for table in tables:
    print(final_table[table].value_counts(), '\n')